In [5]:
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().parent   
sys.path.insert(0, str(PROJECT_ROOT))

print(PROJECT_ROOT)


C:\Users\l1369\Documents\tp2-exploration


In [9]:
from pipeline.main import run_pipeline_geo
from pipeline.storage import load_parquet
from pipeline.quality import QualityAnalyzer


In [10]:
# ==========================================================
#  Exemple d'adresses pour le test du pipeline
# ==========================================================
addresses = [
    "10 Rue de Rivoli 75004 Paris",
    "5 Avenue des Champs Elysées 75008 Paris",
    "1 Place Bellecour 69002 Lyon"
]

In [11]:
# ==========================================================
#  Exécution du pipeline GEO
# ==========================================================
stats = run_pipeline_geo(addresses, max_items=3)

# Charger le résultat final
df = load_parquet(stats["output_path"])
df.head()

# Markdown : Cette cellule exécute l'enrichissement, la transformation, la qualité et le stockage final

🚀 PIPELINE GEO

🌍 ÉTAPE 1 : Enrichissement (géocodage + commune)


Enrichissement GEO: 100%|████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.16s/it]


   💾 Brut: geo_enriched_raw_20251217_132141.json (1.0 KB)

🔧 ÉTAPE 2 : Transformation et nettoyage

📊 ÉTAPE 3 : Analyse de qualité
   Note: A
   Complétude: 100.0%
   Doublons: 0.0%

💾 ÉTAPE 4 : Stockage final
   💾 Parquet: geo_dataset_20251217_132141.parquet (6.4 KB)

✅ PIPELINE GEO TERMINÉ
Durée: 3s
Adresses enrichies: 3
Qualité: A
Fichier: C:\Users\l1369\Documents\tp2-exploration\data\processed\geo_dataset_20251217_132141.parquet


,address,latitude,longitude,score,city,postcode,citycode,commune,population,fetched_at
0,10 Rue de Rivoli 75004 Paris,48.855500,2.360410,0.970275,paris,75004,75104,paris 4e arrondissement,28039,2025-12-17 13:21:39.025450
1,5 Avenue des Champs Elysées 75008 Paris,48.866220,2.316523,0.973239,paris,75008,75108,paris 8e arrondissement,35418,2025-12-17 13:21:40.146311
2,1 Place Bellecour 69002 Lyon,45.758884,4.830916,0.967352,lyon,69002,69382,lyon 2e arrondissement,29880,2025-12-17 13:21:41.260114


In [12]:
# ==========================================================
#  Affichage des métriques de qualité
# ==========================================================
metrics = stats["quality"]
print(f"Note de qualité: {metrics['quality_grade']}")
print(f"Dataset acceptable: {'✅' if metrics['quality_grade'] in ['A','B','C'] else '❌'}")




Note de qualité: A
Dataset acceptable: ✅


In [13]:
# ==========================================================
#  Analyser la qualité avec QualityAnalyzer
# ==========================================================
analyzer = QualityAnalyzer(df)
qa_metrics = analyzer.analyze()
print(f"\n=== Analyse complète ===")
print(f"Note globale : {qa_metrics.quality_grade}")
print(f"Complétude : {qa_metrics.completeness_score*100:.1f}%")
print(f"Doublons : {qa_metrics.duplicates_pct:.1f}%")

# Générer un rapport Markdown
report_path = analyzer.generate_report("exploration_dataset")
print(f"📄 Rapport généré : {report_path}")



=== Analyse complète ===
Note globale : A
Complétude : 100.0%
Doublons : 0.0%
📄 Rapport généré : C:\Users\l1369\Documents\tp2-exploration\data\reports\exploration_dataset_20251217_132149.md


In [15]:
# ==========================================================
# Exploration des anomalies et tendances
# ==========================================================

# Vérification des doublons
if df.duplicated().sum() == 0:
    print("\nAucun doublon dans le dataset.")
else:
    print(f"\nIl existe {df.duplicated().sum()} doublons dans le dataset.")

# Identifier anomalies : score faible (<0.5)
df['anomalie'] = df['score'].apply(lambda x: "Anomalie" if x < 0.5 else "")
print("\nAdresses avec anomalies géographiques :")
print(df[df['anomalie'] == "Anomalie"][['address','score','anomalie']])



Aucun doublon dans le dataset.

Adresses avec anomalies géographiques :
Empty DataFrame
Columns: [address, score, anomalie]
Index: []


In [23]:
import sys
from pathlib import Path
import os

# Racine du projet
PROJECT_ROOT = Path().cwd().parent 
sys.path.insert(0, str(PROJECT_ROOT))
os.chdir(PROJECT_ROOT)
print("Répertoire courant :", os.getcwd())

# Lancer pytest
!python -m pytest ./tests/ -v


Répertoire courant : C:\Users\l1369\Documents\tp2-exploration
============================= test session starts =============================
platform win32 -- Python 3.14.2, pytest-9.0.2, pluggy-1.6.0 -- C:\Users\l1369\Documents\tp2-exploration\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\l1369\Documents\tp2-exploration
configfile: pyproject.toml
plugins: anyio-4.12.0, cov-7.0.0
collecting ... collected 9 items

tests/test_fetchers_geo.py::TestAdresseFetcher::test_fetch_one_valid_address PASSED [ 11%]
tests/test_fetchers_geo.py::TestAdresseFetcher::test_fetch_one_invalid_address PASSED [ 22%]
tests/test_fetchers_geo.py::TestAdresseFetcher::test_fetch_one_empty_address PASSED [ 33%]
tests/test_fetchers_geo.py::TestCommuneFetcher::test_fetch_one_valid_commune PASSED [ 44%]
tests/test_fetchers_geo.py::TestCommuneFetcher::test_fetch_one_invalid_commune PASSED [ 55%]
tests/test_quality_geo.py::TestQualityAnalyzer::test_quality_metrics PASSED [ 66%]
tests/test_transfor

In [24]:
# Avec rapport de couverture
!pytest tests/ -v --cov=pipeline --cov-report=html


============================= test session starts =============================
platform win32 -- Python 3.14.2, pytest-9.0.2, pluggy-1.6.0 -- C:\Users\l1369\Desktop\tp2-exploration\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\l1369\Documents\tp2-exploration
configfile: pyproject.toml
plugins: anyio-4.12.0, cov-7.0.0
collecting ... collected 9 items

tests/test_fetchers_geo.py::TestAdresseFetcher::test_fetch_one_valid_address PASSED [ 11%]
tests/test_fetchers_geo.py::TestAdresseFetcher::test_fetch_one_invalid_address PASSED [ 22%]
tests/test_fetchers_geo.py::TestAdresseFetcher::test_fetch_one_empty_address PASSED [ 33%]
tests/test_fetchers_geo.py::TestCommuneFetcher::test_fetch_one_valid_commune PASSED [ 44%]
tests/test_fetchers_geo.py::TestCommuneFetcher::test_fetch_one_invalid_commune PASSED [ 55%]
tests/test_quality_geo.py::TestQualityAnalyzer::test_quality_metrics PASSED [ 66%]
tests/test_transformer_geo.py::TestDataTransformer::test_remove_duplicates PASSED [